In [ ]:
import os
import re
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import cv2
from sklearn.metrics import jaccard_score 
import glob
import pandas as pd
import random
from skimage import measure 

In [29]:
# 設定隨機種子
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

In [30]:
sys.path.append('C:/Users/user/Jupyter/DeepLabV3/keras-deeplab-v3-plus-master')  
from model import Deeplabv3

In [ ]:
# 路徑設定
PATHS = {
    'Fold1': {
        'train': 'C:/Users/user/Desktop/ML_Work3/Fold1/train',
        'trainannot': 'C:/Users/user/Desktop/ML_Work3/Fold1/trainannot',
        'val': 'C:/Users/user/Desktop/ML_Work3/Fold1/val',
        'valannot': 'C:/Users/user/Desktop/ML_Work3/Fold1/valannot',
        'test': 'C:/Users/user/Desktop/ML_Work3/Fold1/test',
        'testannot': 'C:/Users/user/Desktop/ML_Work3/Fold1/testannot/'
    },
    'Fold2': {
        'train': 'C:/Users/user/Desktop/ML_Work3/Fold2/train',
        'trainannot': 'C:/Users/user/Desktop/ML_Work3/Fold2/trainannot',
        'val': 'C:/Users/user/Desktop/ML_Work3/Fold2/val',
        'valannot': 'C:/Users/user/Desktop/ML_Work3/Fold2/valannot',
        'test': 'C:/Users/user/Desktop/ML_Work3/Fold2/test',
        'testannot': 'C:/Users/user/Desktop/ML_Work3/Fold2/testannot/'
    },
    'Fold3': {
        'train': 'C:/Users/user/Desktop/ML_Work3/Fold3/train',
        'trainannot': 'C:/Users/user/Desktop/ML_Work3/Fold3/trainannot',
        'val': 'C:/Users/user/Desktop/ML_Work3/Fold3/val',
        'valannot': 'C:/Users/user/Desktop/ML_Work3/Fold3/valannot',
        'test': 'C:/Users/user/Desktop/ML_Work3/Fold3/test',
        'testannot': 'C:/Users/user/Desktop/ML_Work3/Fold3/testannot/'
    },
    'Fold4': {
        'train': 'C:/Users/user/Desktop/ML_Work3/Fold4/train',
        'trainannot': 'C:/Users/user/Desktop/ML_Work3/Fold4/trainannot',
        'val': 'C:/Users/user/Desktop/ML_Work3/Fold4/val',
        'valannot': 'C:/Users/user/Desktop/ML_Work3/Fold4/valannot',
        'test': 'C:/Users/user/Desktop/ML_Work3/Fold4/test',
        'testannot': 'C:/Users/user/Desktop/ML_Work3/Fold4/testannot/'
    },
    'Fold5': {
        'train': 'C:/Users/user/Desktop/ML_Work3/Fold5/train',
        'trainannot': 'C:/Users/user/Desktop/ML_Work3/Fold5/trainannot',
        'val': 'C:/Users/user/Desktop/ML_Work3/Fold5/val',
        'valannot': 'C:/Users/user/Desktop/ML_Work3/Fold5/valannot',
        'test': 'C:/Users/user/Desktop/ML_Work3/Fold5/test',
        'testannot': 'C:/Users/user/Desktop/ML_Work3/Fold5/testannot/'
    }
}

# 輸出路徑設定
OUTPUT_DIR = 'C:/Users/user/Desktop/ML_Work3/DeepLabV3+_Result3'
MODEL_SAVE_DIR = os.path.join(OUTPUT_DIR, 'models')
FIGURE_SAVE_DIR = os.path.join(OUTPUT_DIR, 'figures')
RESULTS_SAVE_DIR = os.path.join(OUTPUT_DIR, 'results')

# 建立輸出目錄
for dir_path in [OUTPUT_DIR, MODEL_SAVE_DIR, FIGURE_SAVE_DIR, RESULTS_SAVE_DIR]:
    os.makedirs(dir_path, exist_ok=True)

In [32]:
# 超參數設定
IMG_SIZE = 256 
BATCH_SIZE = 8
EPOCHS = 50
LEARNING_RATE = 0.0005
PIXELS_PER_CM = 72  # 每公分的像素數

In [33]:
def load_image(image_path):
    """載入並前處理圖像"""
    img = cv2.imread(image_path)
    if img is None:
        print(f"無法讀取圖像: {image_path}")
        # 返回一個符合期望 shape 的零數組，而不是 None
        return np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype(np.float32) / 255.0

    if img.ndim == 2: # 灰階圖轉三通道
        img = np.stack((img,)*3, axis=-1)
    elif img.shape[2] == 1: # 單通道圖轉三通道
        img = np.concatenate([img, img, img], axis=-1)
    elif img.shape[2] > 3: # 去掉 alpha 通道等
        img = img[:, :, :3]
    return img

In [34]:
def load_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"無法讀取遮罩: {mask_path}")
        return np.zeros((IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
    mask = (mask > 127).astype(np.float32)  # 確保二值化
    mask = np.expand_dims(mask, axis=-1)
    print(f"Mask min/max: {mask.min()}, {mask.max()}")  # 調試
    return mask

In [36]:
def augment_data(image, mask):
    """對圖像和遮罩進行數據增強"""
    # 確保輸入形狀正確
    if image.ndim != 3 or image.shape[2] != 3:
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        if image.ndim == 2:
            image = np.stack([image, image, image], axis=-1)
        elif image.shape[2] == 1:
            image = np.concatenate([image, image, image], axis=-1)
        elif image.shape[2] > 3:
            image = image[:, :, :3]
    
    # 確保遮罩是正確的形狀
    if mask.ndim != 3 or mask.shape[2] != 1:
        mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
        if mask.ndim == 2:
            mask = np.expand_dims(mask, axis=-1)
        elif mask.shape[2] > 1:
            mask = np.expand_dims(mask[:, :, 0], axis=-1)
    
    # 暫時將遮罩從 (256,256,1) 轉換為 (256,256) 進行處理
    mask_2d = np.squeeze(mask)
    
    # 水平翻轉
    if random.random() > 0.5:
        image = np.fliplr(image)
        mask_2d = np.fliplr(mask_2d)
    
    # 旋轉±10度
    if random.random() > 0.5:
        angle = random.uniform(-10, 10)
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        # 旋轉圖像
        image = cv2.warpAffine(image, rotation_matrix, (w, h), 
                               flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        
        # 旋轉遮罩 (操作2D遮罩)
        mask_2d = cv2.warpAffine(mask_2d, rotation_matrix, (w, h), 
                                flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT)
    
    # 將2D遮罩轉回3D
    mask = np.expand_dims(mask_2d, axis=-1)
    
    # 確保返回的數據是浮點數
    image = image.astype(np.float32)
    mask = mask.astype(np.float32)
    
    # 確保遮罩仍然是二值的
    mask = (mask > 0.5).astype(np.float32)
    
    return image, mask

In [37]:


def match_images_and_masks(image_paths, mask_paths):
    """
    更具彈性的匹配圖像和遮罩路徑函數。
    假設圖像和遮罩檔名中包含一些共同的識別符 (例如數字編號)。
    """
    matched_image_paths = []
    matched_mask_paths = []

    # 從檔名中提取數字識別符的函數示例
    # 您可能需要根據您的檔名格式修改這個正則表達式
    def extract_identifier(filename):
        # 嘗試匹配常見的數字序列，例如 '001', 'image_01', 'frame_123' 等
        # 這個正則表達式會找到檔名中的數字部分
        match = re.search(r'\d+', filename)
        if match:
            return match.group(0) # 返回找到的第一個數字序列
        return os.path.splitext(os.path.basename(filename))[0] # 如果沒有數字，則返回完整檔名（無副檔名）

    # 建立一個從識別符到遮罩路徑的字典
    masks_dict = {}
    for m_path in mask_paths:
        mask_filename = os.path.basename(m_path)
        identifier = extract_identifier(mask_filename)
        if identifier in masks_dict:
            # 處理潛在的識別符衝突，例如一個圖像識別符可能對應到多個遮罩
            # print(f"警告: 遮罩識別符 {identifier} 重複。路徑1: {masks_dict[identifier]}, 路徑2: {m_path}")
            # 這裡可以根據您的情況決定如何處理，例如選擇第一個，或者報錯
            pass
        masks_dict[identifier] = m_path
        # 為了更靈活，也可以考慮去掉 'mask_' 前綴後再存一次 (如果您的遮罩有這種前綴)
        # clean_identifier_for_mask = identifier.replace('mask_', '').replace('_mask', '')
        # if clean_identifier_for_mask != identifier and clean_identifier_for_mask not in masks_dict:
        # masks_dict[clean_identifier_for_mask] = m_path


    num_matched = 0
    for img_path in image_paths:
        img_filename = os.path.basename(img_path)
        identifier = extract_identifier(img_filename)

        if identifier in masks_dict:
            matched_image_paths.append(img_path)
            matched_mask_paths.append(masks_dict[identifier])
            num_matched += 1
        # 可以增加一些調試信息，查看哪些圖像沒有匹配到
        # else:
        #     print(f"圖像 {img_filename} (識別符: {identifier}) 沒有找到對應的遮罩。masks_dict keys: {list(masks_dict.keys())[:10]}")


    print(f"匹配到 {len(matched_image_paths)} 對圖像和遮罩 (原始圖像數: {len(image_paths)}, 原始遮罩數: {len(mask_paths)})")
    if len(matched_image_paths) < len(image_paths) / 2 : # 如果匹配率過低，給出更強的警告
        print(f"警告：匹配到的圖像對數量 ({len(matched_image_paths)}) 遠少於原始圖像數量 ({len(image_paths)})。請仔細檢查檔名規則和 match_images_and_masks 函數中的 extract_identifier 邏輯！")

    return matched_image_paths, matched_mask_paths

In [38]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, mask_paths, batch_size=BATCH_SIZE, augment=False, shuffle=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.indexes = np.arange(len(image_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        current_batch_size = len(batch_indexes) # 處理最後一個 batch 可能不足 batch_size 的情況

        batch_images = np.zeros((current_batch_size, IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)
        batch_masks = np.zeros((current_batch_size, IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)

        for i, idx in enumerate(batch_indexes):
            try:
                image = load_image(self.image_paths[idx])
                mask = load_mask(self.mask_paths[idx])

                if self.augment:
                    image, mask = augment_data(image, mask)

                batch_images[i] = image
                batch_masks[i] = mask
            except Exception as e:
                print(f"處理圖像 {self.image_paths[idx]} 或遮罩 {self.mask_paths[idx]} 時出錯: {e}")
                # 使用空白圖像和遮罩，或採取其他錯誤處理策略
                batch_images[i] = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)
                batch_masks[i] = np.zeros((IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
        return batch_images, batch_masks

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [39]:
def find_ett_endpoint_with_x(mask_squeezed):
    """尋找氣管內管的端點 (y座標) 和對應的 x 座標中點"""
    binary_mask = (mask_squeezed > 0.5).astype(np.uint8)
    white_pixels_y, white_pixels_x = np.where(binary_mask > 0)

    if len(white_pixels_y) == 0:
        return 0, 0  # 如果找不到白色像素

    # 氣管內管的端點是y軸上的最大值（最下方的點）
    endpoint_y = np.max(white_pixels_y)

    # 找到在 endpoint_y 這一行的所有 x 座標
    x_at_endpoint_y = white_pixels_x[white_pixels_y == endpoint_y]
    if len(x_at_endpoint_y) == 0:
        # 如果在最下方那一行沒有白色像素點(理論上不該發生，除非 ETT 是水平的且剛好在圖像邊緣)
        # 可以考慮取整個 ETT 的 x 中點，或者只用 y
        endpoint_x = np.mean(white_pixels_x) if len(white_pixels_x) > 0 else 0
    else:
        endpoint_x = np.mean(x_at_endpoint_y)


    return int(round(endpoint_y)), int(round(endpoint_x))

In [40]:
def calculate_custom_metrics(gt_endpoints_y, pred_endpoints_y, pixels_per_cm=PIXELS_PER_CM):
    """計算自定義評估指標 (僅基於 y 座標)"""
    # 確保輸入是 numpy 數組
    gt_endpoints_y = np.array(gt_endpoints_y)
    pred_endpoints_y = np.array(pred_endpoints_y)

    # 檢查是否有有效的端點數據
    if len(gt_endpoints_y) == 0 or len(pred_endpoints_y) == 0:
        print("警告: 端點列表為空，無法計算自定義指標。")
        return {
            'mean_error_cm': np.nan,
            'acc_within_05cm': np.nan,
            'acc_within_10cm': np.nan
        }
    if len(gt_endpoints_y) != len(pred_endpoints_y):
        print(f"警告: gt_endpoints ({len(gt_endpoints_y)}) 和 pred_endpoints ({len(pred_endpoints_y)}) 長度不匹配。")
        # 可以選擇截斷或填充，或返回 nan
        min_len = min(len(gt_endpoints_y), len(pred_endpoints_y))
        gt_endpoints_y = gt_endpoints_y[:min_len]
        pred_endpoints_y = pred_endpoints_y[:min_len]
        if min_len == 0:
             return {
                'mean_error_cm': np.nan,
                'acc_within_05cm': np.nan,
                'acc_within_10cm': np.nan
            }


    abs_errors_px = np.abs(pred_endpoints_y - gt_endpoints_y)
    abs_errors_cm = abs_errors_px / pixels_per_cm

    mean_error_cm = np.mean(abs_errors_cm)
    accuracy_05cm = 100 * np.mean(abs_errors_px <= (pixels_per_cm / 2))
    accuracy_10cm = 100 * np.mean(abs_errors_px <= pixels_per_cm)

    return {
        'mean_error_cm': mean_error_cm,
        'acc_within_05cm': accuracy_05cm,
        'acc_within_10cm': accuracy_10cm
    }

In [41]:
def iou_score(y_true, y_pred):
    """自定義 IoU (Intersection over Union) 評估指標"""
    y_true = tf.cast(tf.greater(y_true, 0.5), tf.float32)
    y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)

    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3]) - intersection

    iou = (intersection + 1e-10) / (union + 1e-10) # epsilon 避免除以零
    return tf.reduce_mean(iou)


In [42]:
def visualize_improved_results(image, true_mask, pred_mask, save_path=None, model_name="DeepLabV3+"):
    """改進的視覺化函數，統一背景和遮罩顯示"""
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'Microsoft YaHei', 'WenQuanYi Micro Hei']  # 處理中文字體
    plt.rcParams['axes.unicode_minus'] = False

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # 確保遮罩是 2D 的 (H, W)
    if true_mask.ndim > 2:
        true_mask_squeezed = np.squeeze(true_mask)
    else:
        true_mask_squeezed = true_mask
    if pred_mask.ndim > 2:
        pred_mask_squeezed = np.squeeze(pred_mask)
    else:
        pred_mask_squeezed = pred_mask

    # 原始影像
    axes[0].set_title('Image')
    axes[0].imshow(image, cmap='gray')  # X光片通常灰階顯示
    axes[0].axis('off')

    # --- 真實遮罩 ---
    axes[1].set_title('Ground Truth Mask')
    # 背景色 (深藍紫色)
    background_color_gt = [75/255, 0/255, 130/255]  # Indigo-like
    bg_gt = np.ones((IMG_SIZE, IMG_SIZE, 3)) * background_color_gt
    axes[1].imshow(bg_gt)

    # ETT 管用綠色顯示
    gt_display_mask = np.zeros((*true_mask_squeezed.shape, 4))  # RGBA
    gt_display_mask[true_mask_squeezed > 0.5, 1] = 0.8  # Green channel
    gt_display_mask[true_mask_squeezed > 0.5, 3] = 0.9  # Alpha
    axes[1].imshow(gt_display_mask)

    # 標記真實端點 (紅點 + G)
    true_endpoint_y, true_endpoint_x = find_ett_endpoint_with_x(true_mask_squeezed)
    if true_endpoint_y > 0:  # 假設 y=0 代表未找到
        axes[1].plot(true_endpoint_x, true_endpoint_y, 'ro', markersize=8, label='GT Endpoint')
        axes[1].text(true_endpoint_x + 5, true_endpoint_y, 'G', color='red', fontsize=12)
    axes[1].axis('off')

    # --- 預測遮罩 ---
    axes[2].set_title('Predicted Mask')
    # 背景色 (與真實遮罩相同)
    bg_pred = np.ones((IMG_SIZE, IMG_SIZE, 3)) * background_color_gt
    axes[2].imshow(bg_pred)

    # 二值化預測遮罩
    binary_pred_squeezed = (pred_mask_squeezed > 0.5).astype(np.float32)

    # ETT 管用黃色顯示，無論是否檢測到 ETT
    pred_display_mask = np.zeros((*binary_pred_squeezed.shape, 4))  # RGBA
    pred_display_mask[binary_pred_squeezed > 0, 0] = 0.95  # R
    pred_display_mask[binary_pred_squeezed > 0, 1] = 0.95  # G (R+G = Yellow)
    pred_display_mask[binary_pred_squeezed > 0, 3] = 0.9   # Alpha
    axes[2].imshow(pred_display_mask)

    # 標記預測端點 (紅點 + Y)
    pred_endpoint_y, pred_endpoint_x = find_ett_endpoint_with_x(binary_pred_squeezed)
    if pred_endpoint_y > 0:
        axes[2].plot(pred_endpoint_x, pred_endpoint_y, 'ro', markersize=8, label='Pred Endpoint')
        axes[2].text(pred_endpoint_x + 5, pred_endpoint_y, 'Y', color='red', fontsize=12)
    axes[2].axis('off')

    plt.suptitle(f"{model_name} Segmentation Results", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # 調整佈局以容納 suptitle

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close(fig)  # 關閉圖像以釋放記憶體
    else:
        plt.show()

In [43]:
def train_fold(fold_name, paths):
    print(f"=== 訓練 {fold_name} ===")

    train_img_paths = sorted(glob.glob(os.path.join(paths['train'], '*.jpg'))) # 假設圖像是 jpg
    train_mask_paths = sorted(glob.glob(os.path.join(paths['trainannot'], '*.png'))) # 假設遮罩是 png

    val_img_paths = sorted(glob.glob(os.path.join(paths['val'], '*.jpg')))
    val_mask_paths = sorted(glob.glob(os.path.join(paths['valannot'], '*.png')))

    test_img_paths = sorted(glob.glob(os.path.join(paths['test'], '*.jpg')))
    test_mask_paths = sorted(glob.glob(os.path.join(paths['testannot'], '*.png')))

    print(f"訓練資料: 找到 {len(train_img_paths)} 張圖像, {len(train_mask_paths)} 張遮罩")
    print(f"驗證資料: 找到 {len(val_img_paths)} 張圖像, {len(val_mask_paths)} 張遮罩")
    print(f"測試資料: 找到 {len(test_img_paths)} 張圖像, {len(test_mask_paths)} 張遮罩")

    train_img_paths, train_mask_paths = match_images_and_masks(train_img_paths, train_mask_paths)
    val_img_paths, val_mask_paths = match_images_and_masks(val_img_paths, val_mask_paths)
    test_img_paths, test_mask_paths = match_images_and_masks(test_img_paths, test_mask_paths)

    if len(train_img_paths) == 0 or len(val_img_paths) == 0 or len(test_img_paths) == 0:
        print(f"警告: {fold_name} 中沒有足夠的匹配數據。跳過此 Fold。")
        return {
            'fold': fold_name, 'mean_iou': 0, 'mean_error_cm': np.nan,
            'acc_within_05cm': np.nan, 'acc_within_10cm': np.nan
        }, None

    train_generator = DataGenerator(train_img_paths, train_mask_paths, augment=True, shuffle=True)
    val_generator = DataGenerator(val_img_paths, val_mask_paths, augment=False, shuffle=False)

    model = Deeplabv3(input_shape=(IMG_SIZE, IMG_SIZE, 3), classes=1, backbone='xception', weights='pascal_voc')
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy', iou_score])

    model_save_path = os.path.join(MODEL_SAVE_DIR, f'{fold_name}_deeplabv3plus.h5')
    checkpoint = ModelCheckpoint(model_save_path, monitor='val_iou_score', mode='max', save_best_only=True, verbose=1)
    early_stopping = EarlyStopping(monitor='val_iou_score', mode='max', patience=8, verbose=1) # 增加 patience

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=[checkpoint, early_stopping],
        verbose=1
    )

    if os.path.exists(model_save_path): # 確保最佳模型已保存
        print(f"載入最佳模型: {model_save_path}")
        model.load_weights(model_save_path)
    else:
        print(f"警告: 找不到最佳模型檔案 {model_save_path}。將使用訓練結束時的模型進行評估。")


    print(f"=== 評估 {fold_name} ===")
    gt_endpoints_y = []
    pred_endpoints_y = []
    test_ious = []

    if len(test_img_paths) == 0:
        print(f"警告: {fold_name} 沒有測試數據進行評估。")
        return {
            'fold': fold_name, 'mean_iou': 0, 'mean_error_cm': np.nan,
            'acc_within_05cm': np.nan, 'acc_within_10cm': np.nan
        }, history

    for i in range(len(test_img_paths)):
        image = load_image(test_img_paths[i])
        true_mask = load_mask(test_mask_paths[i]) # (H, W, 1)

        # 確保 image 是 (1, H, W, C) 用於預測
        image_batch = np.expand_dims(image, axis=0)
        pred_mask_batch = model.predict(image_batch)
        pred_mask = pred_mask_batch[0] # (H, W, 1)

        # 確保遮罩是 2D 的 (H, W) 用於 find_ett_endpoint_with_x
        true_mask_squeezed = np.squeeze(true_mask)
        pred_mask_squeezed = np.squeeze(pred_mask)

        true_endpoint_y_val, _ = find_ett_endpoint_with_x(true_mask_squeezed)
        pred_endpoint_y_val, _ = find_ett_endpoint_with_x(pred_mask_squeezed)

        gt_endpoints_y.append(true_endpoint_y_val)
        pred_endpoints_y.append(pred_endpoint_y_val)

        # 計算IOU時，確保 true_mask 和 pred_mask 的 shape 和 type 正確
        binary_true_mask_flat = (true_mask_squeezed > 0.5).astype(np.uint8).flatten()
        binary_pred_mask_flat = (pred_mask_squeezed > 0.5).astype(np.uint8).flatten()

        # jaccard_score 需要 non-empty arrays
        if binary_true_mask_flat.size > 0 or binary_pred_mask_flat.size > 0 : # 至少有一個遮罩不是全黑
             iou = jaccard_score(binary_true_mask_flat, binary_pred_mask_flat, zero_division=0)
        else: # 如果兩個都是全黑，可以定義為1或0，這裡設為0
             iou = 0.0
        test_ious.append(iou)


        if i < 10: # 視覺化前10張圖像
            img_basename = os.path.splitext(os.path.basename(test_img_paths[i]))[0]
            save_path = os.path.join(FIGURE_SAVE_DIR, f'{fold_name}_test_img_{img_basename}.png')
            visualize_improved_results(image, true_mask, pred_mask, save_path, model_name="DeepLabV3+")

    custom_metrics = calculate_custom_metrics(gt_endpoints_y, pred_endpoints_y, pixels_per_cm=PIXELS_PER_CM)
    mean_iou = np.mean(test_ious) if test_ious else 0 # 避免空列表導致的錯誤

    results = {
        'fold': fold_name,
        'mean_iou': mean_iou,
        'mean_error_cm': custom_metrics['mean_error_cm'],
        'acc_within_05cm': custom_metrics['acc_within_05cm'],
        'acc_within_10cm': custom_metrics['acc_within_10cm']
    }

    print(f"測試結果 - {fold_name}:")
    print(f"  平均 IoU: {mean_iou:.4f}")
    print(f"  平均誤差(公分): {custom_metrics.get('mean_error_cm', 'N/A'):.4f}")
    print(f"  誤差在0.5公分內準確率: {custom_metrics.get('acc_within_05cm', 'N/A'):.2f}%")
    print(f"  誤差在1.0公分內準確率: {custom_metrics.get('acc_within_10cm', 'N/A'):.2f}%")

    return results, history


In [44]:
# 主函數
def main():
    all_results = []
    histories = {} # 儲存每個 fold 的 history

    for fold_name, paths in PATHS.items():
        try:
            results, history = train_fold(fold_name, paths)
            if results: # 確保 train_fold 返回了有效的結果
                all_results.append(results)
            if history:
                histories[fold_name] = history.history # history.history 包含 loss, acc 等
        except Exception as e:
            print(f"處理 {fold_name} 時發生未預期錯誤: {e}")
            # 可以選擇記錄錯誤並繼續，或者拋出
            import traceback
            traceback.print_exc()
            continue

    if not all_results:
        print("錯誤: 沒有成功處理任何fold。請檢查數據路徑、文件格式和程式碼錯誤。")
        return

    # 過濾掉結果為 None 或包含 NaN 的情況，再計算平均值
    valid_results = [r for r in all_results if r and not any(np.isnan(val) for val in [r.get('mean_iou', np.nan), r.get('mean_error_cm', np.nan)])]

    if not valid_results:
        print("錯誤: 所有 Fold 的結果都無效，無法計算平均績效。")
        return

    mean_results = {
        'fold': 'Average',
        'mean_iou': np.mean([r['mean_iou'] for r in valid_results if 'mean_iou' in r]),
        'mean_error_cm': np.mean([r['mean_error_cm'] for r in valid_results if 'mean_error_cm' in r]),
        'acc_within_05cm': np.mean([r['acc_within_05cm'] for r in valid_results if 'acc_within_05cm' in r]),
        'acc_within_10cm': np.mean([r['acc_within_10cm'] for r in valid_results if 'acc_within_10cm' in r])
    }

    all_results.append(mean_results) # 將平均結果也加入列表

    results_df = pd.DataFrame(all_results)
    results_path = os.path.join(RESULTS_SAVE_DIR, 'deeplabv3plus_results_fixed.csv')
    results_df.to_csv(results_path, index=False)

    print("\n=== 所有Fold平均績效 ===")
    print(f"平均 IoU: {mean_results['mean_iou']:.4f}")
    print(f"平均誤差(公分): {mean_results['mean_error_cm']:.4f}")
    print(f"誤差在0.5公分內準確率: {mean_results['acc_within_05cm']:.2f}%")
    print(f"誤差在1.0公分內準確率: {mean_results['acc_within_10cm']:.2f}%")

    print(f"\n所有結果已保存至: {results_path}")

    # 可以選擇性地繪製每個 fold 的訓練曲線
    for fold_name, history_data in histories.items():
        if history_data:
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)
            plt.plot(history_data['loss'], label='Train Loss')
            if 'val_loss' in history_data:
                plt.plot(history_data['val_loss'], label='Val Loss')
            plt.title(f'{fold_name} - Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()

            plt.subplot(1, 2, 2)
            if 'iou_score' in history_data: # 您的自定義 IoU
                plt.plot(history_data['iou_score'], label='Train IoU')
            if 'val_iou_score' in history_data:
                plt.plot(history_data['val_iou_score'], label='Val IoU')
            plt.title(f'{fold_name} - IoU Score')
            plt.xlabel('Epoch')
            plt.ylabel('IoU')
            plt.legend()

            plt.tight_layout()
            history_fig_path = os.path.join(FIGURE_SAVE_DIR, f'{fold_name}_training_history.png')
            plt.savefig(history_fig_path)
            plt.close() # 關閉圖像
            print(f"{fold_name} 的訓練歷史曲線已保存至: {history_fig_path}")


if __name__ == "__main__":
    main()

=== 訓練 Fold1 ===
訓練資料: 找到 287 張圖像, 287 張遮罩
驗證資料: 找到 47 張圖像, 47 張遮罩
測試資料: 找到 47 張圖像, 47 張遮罩
匹配到 287 對圖像和遮罩 (原始圖像數: 287, 原始遮罩數: 287)
匹配到 47 對圖像和遮罩 (原始圖像數: 47, 原始遮罩數: 47)
匹配到 47 對圖像和遮罩 (原始圖像數: 47, 原始遮罩數: 47)
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Epoch 1/50
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
 1/36 [..............................] - ETA: 11:12 - loss: 2.4370 - accuracy: 0.7534 - iou_score: 0.0076Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max: 0.0, 1.0
Mask min/max